In [24]:
import pandas as pd
import nibabel as nib
import numpy as np
from twisted.web.sux import lenientIdentChars

csv = '../PET_AUTOPSY.csv'
df_autopsias = pd.read_csv(csv)

for indx, row in df_autopsias.iterrows():
    
    ad_pathology = row['ALZHEIMER PATHOLOGY']
    lb_pathology = row['LEWY PATHOLOGY']
    
    if ad_pathology > 1 and lb_pathology <= 1:
        
        group = 'pure-AD'
        
    elif ad_pathology <= 1 and lb_pathology > 1:
        
        group = 'pure-LB'
    
    elif ad_pathology > 1 and lb_pathology > 1:
        
        group = 'Mixed'

    elif ad_pathology <= 1 and lb_pathology <= 1:
        
        group = 'Negative'


    df_autopsias.loc[indx, 'group'] = group
        

print(len(df_autopsias.loc[df_autopsias['group'] == 'pure-AD']))
print(len(df_autopsias.loc[df_autopsias['group'] == 'pure-LB']))


28
8


In [25]:
atlas_path = '../cortex_Harvard_Oxford_orig_atlas.nii.gz'
atlas_img = nib.load(atlas_path)
atlas_data = atlas_img.get_fdata()

atlas_csv = '../Harvard_Oxford.csv'
atlas_df = pd.read_csv(atlas_csv)

dir_imgs = '/Users/jsilva/data/MIA/Autopsia'

for indx, row in df_autopsias.iterrows():
    
    subject = row['SUBJECT_ID']
    fdg_id = row['FDG_ID']
    subj_path = f'{dir_imgs}/{subject}_I{fdg_id}/swfdg_normhist.hdr'
    
    subj_img = nib.load(subj_path)
    subj_data = subj_img.get_fdata()
    
    for indx_a, row_a in atlas_df.iterrows():
        
        roi_name = row_a['ROI_NAME']
        roi_num = row_a['ROI_NUM']
        
        indx_roi = np.where(atlas_data == roi_num)
        roi_mean = np.mean(subj_data[indx_roi])
        
        df_autopsias.loc[indx, f'{roi_name}_mean'] = roi_mean
        

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


KeyboardInterrupt: 

In [3]:
df_autopsias.to_csv('roi_values.csv', index=False)

In [11]:
# Cada una de las regiones
# Two-sample t-test (para cada ROI) y vamos a coger las regiones con la mayor significancia.
from scipy.stats import ttest_ind

group_ad = df_autopsias.loc[df_autopsias['group'] == 'pure-AD']
group_lb = df_autopsias.loc[df_autopsias['group'] == 'pure-LB']

preserved_rois = []

for roi in group_lb.columns[9:]:
    
    t,p = ttest_ind(group_ad[roi], group_lb[roi])
    if p<0.005:
        print(roi,': ',p)
        preserved_rois.append(roi)

len(preserved_rois)

Inferior_Temporal_Gyrus_anterior_division_mean :  0.0008766686428361001
Intracalcarine_Cortex_mean :  0.001631662339819495
Supracalcarine_Cortex_mean :  0.00484164283053538


3

In [18]:
df_train = df_autopsias.loc[df_autopsias['group'] != 'Negative']
df_train = df_train.loc[df_train['group'] != 'Mixed']

X=df_train[preserved_rois]
y=df_train['group']

# Regresion logistica
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

model = LogisticRegression(class_weight='balanced')
model.fit(X, y)

y_pred = model.predict(X)
conf_matrix = confusion_matrix(y, y_pred)
accuracy_score = accuracy_score(y, y_pred)
print(accuracy_score)
conf_matrix

#class sklearn.linear_model.LogisticRegression(penalty='l2', *, dual=False, tol=0.0001, C=1.0, fit_intercept=True,#intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, #multi_class='deprecated', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)


0.8611111111111112


array([[24,  4],
       [ 1,  7]])

In [20]:
from sklearn.svm import LinearSVC

model_svm = LinearSVC(class_weight='balanced')
model_svm.fit(X, y)

y_pred = model_svm.predict(X)
conf_matrix = confusion_matrix(y, y_pred)
#accuracy_score = accuracy_score(y, y_pred)
#print(accuracy_score)
conf_matrix


/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


array([[25,  3],
       [ 1,  7]])

In [21]:
import pickle

# save the model to disk
filename = 'SVM_model.sav'
pickle.dump(model_svm, open(filename, 'wb'))

In [23]:
X.columns

Index(['Inferior_Temporal_Gyrus_anterior_division_mean',
       'Intracalcarine_Cortex_mean', 'Supracalcarine_Cortex_mean'],
      dtype='object')